<a href="https://colab.research.google.com/github/vaghasiatirth24/Machine-Learning/blob/main/ML_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_dataset=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

In [ ]:
print(train_dataset.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [ ]:
test_data["Sex"]=pd.get_dummies(test_data["Sex"])
train_dataset["Sex"]=pd.get_dummies(train_dataset["Sex"])

In [ ]:
train_dataset['Age'].fillna(train_dataset['Age'].median(), inplace = True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)

train_dataset['Fare'].fillna(train_dataset['Fare'].median(), inplace = True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace = True)

In [ ]:
combine=[train_dataset,test_data]
for dataset in combine:
  dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(train_dataset['Title'], train_dataset['Sex'])

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    


In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [ ]:
train_dataset['Age_bin'] = pd.cut(train_dataset['Age'], bins=[0,12,20,40,120], labels=['Children','Teenage','Adult','Elder'])
test_data['Age_bin'] = pd.cut(test_data['Age'], bins=[0,12,20,40,120], labels=['Children','Teenage','Adult','Elder'])

In [ ]:
train_dataset['Fare_bin'] = pd.cut(train_dataset['Fare'], bins=[0,7.91,14.45,31,120], labels=['Low_fare','median_fare','Average_fare','high_fare'])
test_data['Fare_bin'] = pd.cut(test_data['Fare'], bins=[0,7.91,14.45,31,120], labels=['Low_fare','median_fare','Average_fare','high_fare'])


In [ ]:
print(train_dataset.iloc[0])

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                                  0
Age                                 22
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Title                                1
Age_bin                          Adult
Fare_bin                      Low_fare
Name: 0, dtype: object


In [ ]:
print(test_data.iloc[0])

PassengerId                 892
Pclass                        3
Name           Kelly, Mr. James
Sex                           0
Age                        34.5
SibSp                         0
Parch                         0
Ticket                   330911
Fare                     7.8292
Cabin                       NaN
Embarked                      Q
Title                         1
Age_bin                   Adult
Fare_bin               Low_fare
Name: 0, dtype: object


In [ ]:
train_dataset = pd.get_dummies(train_dataset, columns = ["Age_bin","Embarked"],prefix=["Age_type","Em_type"])


In [ ]:
test_data = pd.get_dummies(test_data, columns = ["Age_bin","Embarked"],prefix=["Age_type","Em_type"])

In [ ]:
train_dataset = pd.get_dummies(train_dataset, columns = ["Fare_bin"],prefix=["Fare_type"])
test_data = pd.get_dummies(test_data, columns = ["Fare_bin"],prefix=["Fare_type"])

In [ ]:
train_dataset = train_dataset.drop(['Name','PassengerId','Cabin','Ticket'], axis=1)
test_data = test_data.drop(['Name','PassengerId','Cabin','Ticket'], axis=1)

In [ ]:
train_dataset = train_dataset.drop(['Fare','Age'], axis=1)
test_data = test_data.drop(['Fare','Age'], axis=1)

In [ ]:
print(train_dataset.iloc[0])

Survived                  0
Pclass                    3
Sex                       0
SibSp                     1
Parch                     0
Title                     1
Age_type_Children         0
Age_type_Teenage          0
Age_type_Adult            1
Age_type_Elder            0
Em_type_C                 0
Em_type_Q                 0
Em_type_S                 1
Fare_type_Low_fare        1
Fare_type_median_fare     0
Fare_type_Average_fare    0
Fare_type_high_fare       0
Name: 0, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
f = train_dataset.drop("Survived",axis=1)
t = train_dataset["Survived"]
X_train,X_test,y_train,y_test = train_test_split(f,t,test_size=0.3,random_state=1)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((623, 16), (268, 16), (623,), (268,))

In [ ]:
print(X_train.iloc[0])

Pclass                    3
Sex                       1
SibSp                     0
Parch                     0
Title                     2
Age_type_Children         0
Age_type_Teenage          1
Age_type_Adult            0
Age_type_Elder            0
Em_type_C                 1
Em_type_Q                 0
Em_type_S                 0
Fare_type_Low_fare        0
Fare_type_median_fare     0
Fare_type_Average_fare    1
Fare_type_high_fare       0
Name: 114, dtype: int64


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
model= DecisionTreeClassifier(criterion='gini', min_samples_split=10,min_samples_leaf=1,max_features='auto')
t1=model.fit(X_train,y_train)
prediction_tree=model.predict(X_test)
print('The accuracy of the DecisionTree Classifier is',round(accuracy_score(prediction_tree,y_test)*100,2))
kfold = KFold(n_splits=5, random_state=1)
result_tree=cross_val_score(model,f,t,cv=5,scoring='accuracy')
print('The cross validated score for Decision Tree classifier is:',round(result_tree.mean()*100,2))
y_pred = cross_val_predict(model,f,t,cv=5)
tree.plot_tree(t1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(criterion='gini', n_estimators=1000,
                             min_samples_split=10,min_samples_leaf=1,
                             max_features='auto',oob_score=True,
                             random_state=1,n_jobs=-1)
model.fit(X_train,y_train)
prediction_rm=model.predict(X_test)
print('The accuracy of the Random Forest Classifier is', round(accuracy_score(prediction_rm,y_test)*100,2))
kfold = KFold(n_splits=5, random_state=1)
result_rm=cross_val_score(model,f,t,cv=5,scoring='accuracy')
print('The cross validated score for Random Forest Classifier is:',round(result_rm.mean()*100,2))
y_pred = cross_val_predict(model,f,t,cv=5)

The Accuracy of the model
The accuracy of the Random Forest Classifier is 78.73


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


The cross validated score for Random Forest Classifier is: 82.6
